In [54]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [55]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_dropped = application_df.drop(['EIN', 'NAME'], axis=1)
application_df_dropped.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [56]:
# Determine the number of unique values in each column.
application_df_dropped.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [57]:
# Looking at APPLICATION_TYPE value counts for binning
application_df_dropped['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [58]:
# Viewing application types list for cutoff
application_df_dropped['APPLICATION_TYPE'].value_counts().index[9:]

Index(['T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17'], dtype='object', name='APPLICATION_TYPE')

In [59]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_cutoff = application_df_dropped['APPLICATION_TYPE'].value_counts().index[9:]

# Replace in dataframe
for app in application_types_cutoff:
    application_df_dropped['APPLICATION_TYPE'] = application_df_dropped['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_dropped['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [60]:
# Looking at CLASSIFICATION value counts >1 for binning
application_df_dropped['CLASSIFICATION'].value_counts().loc[lambda x : x > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [61]:
# Viewing classifications list to cutoff
application_df_dropped['CLASSIFICATION'].value_counts().loc[lambda x : x > 1].index[15:]

Index(['C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120', 'C1500',
       'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278', 'C1235', 'C1237',
       'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710',
       'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256'],
      dtype='object', name='CLASSIFICATION')

In [62]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_cutoff = application_df_dropped['CLASSIFICATION'].value_counts().loc[lambda x : x > 1].index[15:]

# Replace in dataframe
for classification in classifications_cutoff:
    application_df_dropped['CLASSIFICATION'] = application_df_dropped['CLASSIFICATION'].replace(classification,"Other")
    
# Check to make sure binning was successful
application_df_dropped['CLASSIFICATION'].value_counts().loc[lambda x : x > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      365
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: count, dtype: int64

In [63]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df_dropped, dtype=float)
dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [64]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.copy()
X.drop('IS_SUCCESSFUL', axis=1, inplace=True)
y = dummies_df['IS_SUCCESSFUL']
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)

In [65]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [66]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer_1 = 4
hidden_nodes_layer_2 = 6
hidden_nodes_layer_3 = 8

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer_1, input_dim=number_input_features, activation='relu'))
# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer_2, activation='relu'))
# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer_3, activation='relu'))
# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 4)                 324       
                                                                 
 dense_13 (Dense)            (None, 6)                 30        
                                                                 
 dense_14 (Dense)            (None, 8)                 56        
                                                                 
 dense_15 (Dense)            (None, 1)                 9         
                                                                 
Total params: 419 (1.64 KB)
Trainable params: 419 (1.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [67]:
# Compile the model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [68]:
# Train the model
nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100


804/804 [==============================] - 1s 597us/step - loss: 0.6328 - accuracy: 0.6899
Epoch 2/100
804/804 [==============================] - 0s 607us/step - loss: 0.5837 - accuracy: 0.7223
Epoch 3/100
804/804 [==============================] - 0s 600us/step - loss: 0.5715 - accuracy: 0.7247
Epoch 4/100
804/804 [==============================] - 0s 607us/step - loss: 0.5659 - accuracy: 0.7256
Epoch 5/100
804/804 [==============================] - 0s 600us/step - loss: 0.5624 - accuracy: 0.7261
Epoch 6/100
804/804 [==============================] - 0s 594us/step - loss: 0.5597 - accuracy: 0.7261
Epoch 7/100
804/804 [==============================] - 0s 598us/step - loss: 0.5580 - accuracy: 0.7283
Epoch 8/100
804/804 [==============================] - 0s 594us/step - loss: 0.5556 - accuracy: 0.7311
Epoch 9/100
804/804 [==============================] - 0s 596us/step - loss: 0.5543 - accuracy: 0.7329
Epoch 10/100
804/804 [==============================] - 0s 594us/step - loss: 0.5533 

In [69]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5563 - accuracy: 0.7308 - 184ms/epoch - 688us/step
Loss: 0.5563403964042664, Accuracy: 0.7308454513549805


In [70]:
# Export our model to HDF5 file
nn_model.save('../Models/AlphabetSoupCharity_Optimization_3.h5')

c:\Users\Mario Martinez\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
